In [1]:
import os
import pandas as pd

In [2]:
import json
with open(os.path.join("data_fixes", "seznamy.json"), encoding="utf-8") as seznamy:
    seznamy = json.loads(seznamy.read())

In [3]:
profese = seznamy["profese"]

In [4]:
def plose_jednoduse(x):
    seznam = []
    for y in x:
        if y == None:
            pass
        if isinstance(y, str):
            if y not in seznam:
                seznam.append(y)
        if isinstance(y, list):
            for z in y:
                if z not in seznam:
                    seznam.append(z)
    return(seznam)

### Entity s malým písmenem na začátku, když to není titul nebo hodnost

In [5]:
df = pd.read_json(os.path.join("data", "filmy.json"))

In [6]:
df["Lidé"] = df[profese].values.tolist()
df["Lidé"] = df["Lidé"].apply(lambda x: plose_jednoduse(x))
df = df.explode("Lidé")

In [7]:
lide = df["Lidé"].dropna().drop_duplicates()

In [8]:
lide

0               Jan Kříženecký (127419)
1       Josef Šváb-Malostranský (55291)
2                Ferdinand Gýra (60351)
2              Marie Kříženecká (99776)
18                   František Josef I.
                     ...               
4554                       Michal Fuchs
4554                          Jan Veitz
4554                       Jakub Reindl
4554                  Dušan Kozák (zvuk
4554                     Michal Šťastný
Name: Lidé, Length: 74378, dtype: object

In [9]:
lide = lide[~lide.str.contains("(arch\.|ak\. sochař|doc\.|dr\.|genmj\.|genplk\.|genpor\.|ing\.|inž\.|JUDr\.|kapitán|kpt\.|mjr\.|npor\.|plk\.|pplk\.|prof\.)",regex=True)]

C:\Users\micha\AppData\Local\Temp\ipykernel_4468\3232200283.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lide = lide[~lide.str.contains("(arch\.|ak\. sochař|doc\.|dr\.|genmj\.|genplk\.|genpor\.|ing\.|inž\.|JUDr\.|kapitán|kpt\.|mjr\.|npor\.|plk\.|pplk\.|prof\.)",regex=True)]


In [10]:
mask = lide.str[0].str.islower()

In [11]:
result = lide[mask]

In [12]:
result.sample(1000).drop_duplicates().to_list()

['manželé Mlčákovi',
 'pes Doly',
 'pozadí 122398',
 'horníci Dolu Petr Cinger Ostrava',
 'sýkora modřinka',
 'kolektiv pracovníků Ústavu experimentální biologie a genetiky Československé akademie věd',
 'obyvatelé české vesnice Gernik v rumunském Banátu',
 'supervize a mix zvuku',
 'mravenec Myrmica rugulosa',
 'střih a mix titulní písně)',
 'lidoví tanečníci z Moravského Slovácka',
 'loudmark.cz',
 'záznam ruchů)',
 'pan Polzar',
 'špaček obecný',
 'skupina historického šermu Valdštejnové',
 'děkanát Filozofické fakulty Univerzity Palackého Olomouc',
 'kanec Honza',
 'pes Dorka',
 'mistři Modřanských strojíren n. p.',
 'taneční duo Farinti (75558)',
 'medvěd Džony',
 'obyvatelé obce Zduchovice',
 'společnost pro kulturní aktivitu',
 'obyvatelé Vsetína',
 'plšík lískový',
 'krabi mangrovoví',
 'pelikán kadeřavý',
 'příslušníci Čs. lidové armády',
 'členové Dřevěného divadla',
 'skupina historického šermu Markus M.',
 'obyvatelé obcí Blažejovice, Martinice a Bolechov',
 'dvacet dva ovc

In [13]:
nejsou_lide = result.drop_duplicates().to_list()

### Firmy označené jako firmy

In [14]:
persony = pd.read_json(os.path.join("data", "persony.json"))

In [15]:
persony = persony[persony["Osoba"] !="fyzická"]

In [16]:
firmy = persony["Jméno"].drop_duplicates().to_list()
firmy

['kino Světozor (100019)',
 'AD WOOD, s.r.o. (100038)',
 'Hypermarket Film s.r.o. (100039)',
 'Punk Film, s.r.o. (100076)',
 'FILM FACTORY, s.r.o. (100137)',
 'FILM FACTORY CZ, s.r.o. (100138)',
 'Pegasos Film (100196)',
 'SisaArt (100197)',
 'Asociácia slovenských filmových klubov – ASFK (100216)',
 'Starorežná Prostějov, k.s. (100219)',
 'Agentura Tomáš Zůbek (100221)',
 'Radka Production s.r.o. (100236)',
 'MAUR film s.r.o. (100237)',
 'Here and Now Production (100262)',
 'Open City Films (100263)',
 'Česká ergonomická společnost, z.s. (100279)',
 'NATIONALFILM spol. s r.o. (100296)',
 '@ARMZ Inc. (100336)',
 'Pallas Film GmbH (100396)',
 'HOO DOO Production s.r.o. (100416)',
 'Soundsquare, a.s. (100417)',
 'Cabiria s. r. o. (100438)',
 'RUNTEAM II Ltd. (100441)',
 'R.P. FILMS (100442)',
 'Motion FX (100443)',
 'VFX ÉCLAIR (100444)',
 'Oliver Twist Productions LLp (100445)',
 'A-NEWS, s.r.o. (100497)',
 'Ministerstvo zemědělství, lesního a vodního hospodářství ČSSR (100537)',
 'Diva

In [17]:
for f in firmy:
    nejsou_lide.append(f)

### Firmy neoznačené jako firmy

In [18]:
nelidske_nazvy = ["agentu","producti","effect","agency","studio","a.s.","a. s.","s.r.o.","společnost","televiz"]

In [19]:
for v in lide:
    if isinstance (v, str):
        for n in nelidske_nazvy:
            if n in v:
                if n not in nejsou_lide:
                    nejsou_lide.append(v)

In [20]:
len(nejsou_lide)

4372

In [21]:
with open(os.path.join("data_fixes", "nelide.json"), "w+", encoding="utf-8") as nelide:
    json.dump(nejsou_lide, nelide)